<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Configure-Azure-ML-workspace" data-toc-modified-id="Configure-Azure-ML-workspace-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Configure Azure ML workspace</a></span></li><li><span><a href="#HyperDrive-Pipeline" data-toc-modified-id="HyperDrive-Pipeline-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>HyperDrive Pipeline</a></span><ul class="toc-item"><li><span><a href="#Create-Resources-for-Training-Experiments" data-toc-modified-id="Create-Resources-for-Training-Experiments-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create Resources for Training Experiments</a></span></li><li><span><a href="#Hyperparameter-Tunning" data-toc-modified-id="Hyperparameter-Tunning-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Hyperparameter Tunning</a></span><ul class="toc-item"><li><span><a href="#Parameter-sampler" data-toc-modified-id="Parameter-sampler-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Parameter sampler</a></span></li><li><span><a href="#Early-Termination-Policy" data-toc-modified-id="Early-Termination-Policy-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Early Termination Policy</a></span></li><li><span><a href="#Create-a-SKLearn-Estimator" data-toc-modified-id="Create-a-SKLearn-Estimator-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Create a SKLearn Estimator</a></span></li><li><span><a href="#Create-a-HyperDriveConfig" data-toc-modified-id="Create-a-HyperDriveConfig-3.2.4"><span class="toc-item-num">3.2.4&nbsp;&nbsp;</span>Create a <a href="https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py" target="_blank">HyperDriveConfig</a></a></span></li></ul></li></ul></li><li><span><a href="#AutoML-Run" data-toc-modified-id="AutoML-Run-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>AutoML Run</a></span><ul class="toc-item"><li><span><a href="#Create-Dataset" data-toc-modified-id="Create-Dataset-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Create Dataset</a></span></li><li><span><a href="#Inspect-Dataset" data-toc-modified-id="Inspect-Dataset-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Inspect Dataset</a></span></li><li><span><a href="#Clean-and-Split-Dataset" data-toc-modified-id="Clean-and-Split-Dataset-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Clean and Split Dataset</a></span></li><li><span><a href="#Configure-Experiment" data-toc-modified-id="Configure-Experiment-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Configure Experiment</a></span></li><li><span><a href="#Submitting-Training-Experiment" data-toc-modified-id="Submitting-Training-Experiment-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Submitting Training Experiment</a></span></li><li><span><a href="#Monitor-using-Widget" data-toc-modified-id="Monitor-using-Widget-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Monitor using <code>Widget</code></a></span></li><li><span><a href="#Retrieve-and-Save-Best-Model" data-toc-modified-id="Retrieve-and-Save-Best-Model-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>Retrieve and Save Best Model</a></span></li></ul></li><li><span><a href="#Cleaning-Up-Cluster" data-toc-modified-id="Cleaning-Up-Cluster-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Cleaning Up Cluster</a></span></li></ul></div>

# Introduction

# Configure Azure ML workspace

For this project we will be using an Azure Machine Learning Notebook VM, therefore we can skip setting up the environment.

To start we need to initialize our workspace and create a Azule ML experiment.

In [ ]:
from azureml.core import Workspace, Experiment

# Initialize a workspace object for an existing Azure Machine Learning Workspace
ws = Workspace.get("quick-starts-ws-127145")

# Create a experiment
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

# HyperDrive Pipeline

## Create Resources for Training Experiments

Now that we have initialized our workspace and created our experiment, it is time to define our resources.

In this section you will create default compute clusters for use by the notebook and any other necessary operations we need.

In order to create a cluster we need to specify a compute configuration that defines the `type of machine` to be used and the `scalability behaviors`. Also, it is necessary to define the name of the cluster which must be unique within the workspace. This name is used to address the cluster later.

For this project we use a CPU cluster with following parameters:

* `type of the machine`:

    * `vm_size`: Defines the size of the virtual machine. We use here "STANDARD_D2_V2" (more details [here](https://docs.microsoft.com/en-us/azure/cloud-services/cloud-services-sizes-specs#dv2-series))

* `Scalability behaviors`:

    * `min_nodes`: Sets minimun size of the cluster. Setting the minimum to 0 the cluster will shut down all nodes while not in use. If you use another value you are able to have faster start-up times, but you will also be billed when the cluster is not in use.

    * `max_nodes`: Sets the maximun size of the cluster. Larger number allows for more concurrency and a greater distributed processing of scale-out jobs.



In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Define CPU cluster name
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0, # when innactive
                                                           max_nodes=4) # when busy

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
# from azureml.core.compute import ComputeTarget, AmlCompute

# # TODO: Create compute cluster
# # Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# # max_nodes should be no greater than 4.

# # name of CPU cluster
# amlcompute_cluster_name = "cpu-cluster"
# vm_size = 'STANDARD_D2_V2'
# max_nodes = 4

# compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,max_nodes=max_nodes)
# aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

## Hyperparameter Tunning

### Parameter sampler

In this example using HyperDrive we use [`random sampling`](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.randomparametersampling?view=azure-ml-py) to try different configuration sets of hyperparameters to maximize the chosen primary metric, accuracy. The function `choice` specify a discrete set of options to sample from.

The hyperparameters and metric used are defined in the script `train.py`.

### Early Termination Policy

This saves us from continuing to explore hyperparameters that don't show promise of helping reach our target metric.

An early termination policy help us improving computational efficiency by terminating poorly performing runs.

The `early termination policy` we used [`Bandit Policy`]( https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?preserve-view=true&view=azure-ml-py#&preserve-view=truedefinition ). This policy is based on `slack factor/slack amount` and `evaluation interval`. Bandit terminates runs where the primary metric is not within the specified slack factor/slack amount compared to the best performing run.

This allows more aggressive savings than Median Stopping policy if we apply a smaller allowable slack.

Parameter `slack_factor` which is the slack allowed with respect to the best performing training run, need to be defined while `evaluation_interval` and `delay_interval` are optional.

`evaluation_interval` says when the policy is applied. If the `evaluation_interval` is not defined the default value is one, i.e., policy is applied every time the training script reports the primary metric.

Specifying `delay_interval` avoids premature termination of training runs by allowing all configurations to run for a minimum number of intervals. If specified, the policy applies every multiple of evaluation_interval that is greater than or equal to delay_evaluation.

For example, in our example, by applying the Bandit policy with `slack_factor = 0.1`, `evaluation_interval=2`, `delay_evaluation=5` the early termination policy is applied at every other time interval when metrics are reported, starting at evaluation interval 5. Any run whose primary metric falls outside of the top 10% range, Azure ML terminate the job.

### Create a SKLearn Estimator

[SKLearn Class](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.sklearn.sklearn?view=azure-ml-py) creates an estimator for training in Scikit-learn experiments.

### Create a [HyperDriveConfig](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py)

Now we are ready to configure a run configuration object. 

As parameters we inform `parameter sampler`, `early termination policy`, and `estimator` that we just configured. We also specify the primary metric `Accuracy` that's recorded in your training runs and we tell the service that we want to maximize this value.  

Moreover, we set the `number of samples` to 20, and `maximal concurrent job` to 4, which is the same as the number of nodes in our computer cluster.


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler

ps = RandomParameterSampling({
    '--C': choice(0.01, 0.1, 0.2, 0.5, 0.7, 1.0)
    '--max_iter': choice(range(10,110,10))
    }
)


# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, # specifies the allowable slack as a ratio
                      evaluation_interval=2, # frequency for applying the policy
                      delay_evaluation=5) # delays the first policy evaluation for a specified number of intervals


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

est = SKLearn( 
    source_directory='./', # directory containing experiment configuration files (train.py)
    compute_target=cpu_cluster, # compute target where training will happen
    vm_size=vm_size, # VM size of the compute target
    vm_priority='lowpriority' # VM priority of the compute target (default value is 'dedicated')
    entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###

# REF: https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py

hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=early_termination_policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4
                                    )



In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config = hyperdrive_config)

# Watch the progress of the run with a Jupyter widget. 

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)


In [ ]:
import joblib

# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])

best_run

In [ ]:
# get name of files of best_run
best_run.get_file_names()

In [ ]:
# save the model, i.e., output file of best_run
model = best_run.register_model(model_name='model_hd', model_path='outputs/model.joblib')

# AutoML Run

Now we use the same dataset to obtain a model by running AutoML.

## Create Dataset

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory

ds = TabularDatasetFactory.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

## Inspect Dataset

In [1]:
# inspecting data

ds.head()

NameError: name 'ds' is not defined

In [ ]:
ds.info()

In [ ]:
ds.y.value_counts(normalize=True)

## Clean and Split Dataset

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from sklearn.model_selection import train_test_split

# split data into train and test sets - 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=123, stratify = y)

## Configure Experiment 

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="Accuracy",
    training_data=x_train,
    label_column_name=y_train,
    n_cross_validations=5)

## Submitting Training Experiment

In [2]:
# Submit your automl run

experiment = Experiment(ws, "automl_experiment")
automl_run = experiment.submit(config=automl_config, show_output=True)
automl_run


## Monitor using `Widget`

Once more we make use of `widget`. This time to explore the results obtained by using AutoML.

In [ ]:
# from azureml.widgets import RunDetails
RunDetails(automl_run).show()

automl_run.wait_for_completion(show_output=True)

## Retrieve and Save Best Model

Below we select the best model from all the training iterations using get_output method.


In [ ]:
# Retrieve model

best_run, fitted_model = automl_run.get_output()

# get name of files of best_run
best_run.get_file_names()

In [ ]:
# save best model
best_automl_run.register_model(model_name = "automl.pkl", model_path = './outputs/')
print(best_model._final_estimator)

In [ ]:
fitted_model.steps

### YOUR CODE HERE ###

# Cleaning Up Cluster

In [ ]:
compute_target.delete()